In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [47]:
from tqdm import tqdm

## 读入数据

In [11]:
pwd = '/home/blu/workspace/graduate_project/STGCN/beijing_traffic/mini_data/'

#train
x_train = np.load(pwd + "x_train.npy")
y_train = np.load(pwd + "y_train.npy")
#val
x_val = np.load(pwd + "x_val.npy")
y_val = np.load(pwd + "y_val.npy")

In [12]:
x_train_mean = x_train.mean()
x_train_std = x_train.std()
y_train_mean = y_train.mean()
y_train_std = y_train.std()
x_train_uni = (x_train - x_train.mean())/x_train_std
y_train_uni = (y_train - y_train.mean())/y_train_std

x_val_mean = x_val.mean()
x_val_std = x_val.std()
y_val_mean = y_val.mean()
y_val_std = y_val.std()
x_val_uni = (x_val - x_val.mean())/x_val_std
y_val_uni = (y_val - y_val.mean())/y_val_std

In [13]:
print("x_train_uni:", x_train_uni.shape, "y_train_uni:", y_train_uni.shape)
print("x_val_uni:", x_val_uni.shape, "y_val_uni:", y_val_uni.shape)

x_train_uni: (4090, 12, 216) y_train_uni: (4090, 1, 216)
x_val_uni: (584, 12, 216) y_val_uni: (584, 1, 216)


In [54]:
x_train_list = []

In [55]:
for i in tqdm(range(x_train_uni.shape[0])):
    x_train_list.append(x_train_uni[i][np.newaxis,:])

100%|██████████| 4090/4090 [00:00<00:00, 479543.32it/s]


In [92]:
x_val_list = []
for i in tqdm(range(x_val_uni.shape[0])):
    x_val_list.append(x_val_uni[i][np.newaxis,:])

100%|██████████| 584/584 [00:00<00:00, 260776.49it/s]


In [131]:
y_train_uni[0].shape

(1, 216)

## 导入模型

In [135]:
num_units = 216
num_outputs = 216
encoder_emb_inp = tf.keras.Input(shape=(12, 216), name='Input')

encoder = tf.keras.layers.LSTM(num_units, return_state=True, return_sequences = True, input_length = 12)
encoder_outputs, state_h, state_c = encoder(encoder_emb_inp)
encoder_state = [state_h, state_c]

print("encoder_shape = ", encoder_outputs.shape)

decoder_emb_inp = encoder_outputs

# Sampler
sampler = tfa.seq2seq.sampler.TrainingSampler()

# Decoder
decoder_cell = tf.keras.layers.LSTMCell(num_units)
projection_layer = tf.keras.layers.Dense(num_outputs)
decoder = tfa.seq2seq.BasicDecoder(decoder_cell, sampler, output_layer=projection_layer)

outputs, _, _ = decoder(decoder_emb_inp, initial_state=encoder_state, sequence_length=[12])

logits = outputs.rnn_output
print("logits_shape = ", logits.shape)

model = tf.keras.Model(inputs=encoder_emb_inp, outputs=logits, name='seq2seq')
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())

encoder_shape =  (None, 12, 216)


AttributeError: 'BasicDecoderOutput' object has no attribute 'shape'

模型训练与验证

In [133]:
history = model.fit(x_train_list, 
                    y_train_uni,
                    batch_size=1,
                    epochs=3,
                    validation_data=(x_val_list,
                                     y_val_uni))

ValueError: Input arrays should have the same number of samples as target arrays. Found 1 input samples and 4090 target samples.